In [39]:
import sys
sys.path.append('.')
sys.path.append('..')

from subreddit_frequency import load_dataframe_from_jsonl
from tqdm import tqdm
tqdm.pandas()

import seaborn as sns
from datetime import datetime
sns.set('paper')

from ipywidgets import interact
import pandas as pd

In [25]:
comments_df = load_dataframe_from_jsonl("../data/RC_2019-08_AmItheAsshole.dump")
submissions_df = load_dataframe_from_jsonl("../data/RS_2019-08_AmItheAsshole.dump")

1454892it [00:45, 32194.47it/s]
34476it [00:01, 24617.49it/s]


Preprocess the comment to have:
* the id of the previous post that it is a comment to
* the absolute value of the score

In [69]:
def get_comments_from_id(df, parent_id):
    cols = ['author_flair_text', 'stickied', 'author', 'body', 'score', 'score_abs', 'judgement']
    return df[df.prev_id == parent_id][cols]

In [70]:
def determine_AH(body):
    """Determines if poster thinks asshole or not asshole."""
    if body.startswith("YTA"):
        return "YTA"
    elif body.startswith("ESH"):
        return "ESH"
    elif body.startswith("NAH"):
        return "NAH"
    else:
        return "UNK"

In [71]:
comments_df['prev_id'] = comments_df.parent_id.map(lambda x: x.split('_')[-1])
comments_df['score_abs'] = comments_df.score.map(abs)
comments_df['judgement'] = comments_df.body.map(determine_AH)
submissions_df['timestamp'] = submissions_df.created_utc.map(datetime.fromtimestamp)
submissions_df = submissions_df.sort_values('num_comments', ascending=False)

Only get the most popular titles with the highest number of comments.

In [72]:
pd.set_option('display.max_rows', 500)
good_submissions_df = submissions_df[['title', 'num_comments', 'id']].head(2000)

In [73]:
display(list(good_submissions_df.head().title))
display(list(good_submissions_df.head().id))

['AITA for refusing to pay back my cousin after my baby ruined her blouse?',
 'AITA for wanting my girlfriend to wear makeup and take care of body hair?',
 'AITA for telling a friend’s friend that he couldn’t keep the “jackpot” that he hit on my antique slot machine? (About $700)',
 'WIBTA if I told a close family friend that her husband cheated on her 4 years ago?',
 'AITA for catfishing my underaged sister on Tinder and humiliating her in order to teach her a lesson?']

['cw43oc', 'cqin60', 'cm0bft', 'cvlkut', 'cn7li5']

In [86]:
def get_label_from_comments(df):
    try:
        return df[df.judgement != "UNK"].groupby('judgement').score.sum().idxmax()
    except ValueError:
        return "UNK"
    
def get_label_from_submission(submission_id):
    df = get_comments_from_id(comments_df, submission_id)
    return get_label_from_comments(df)

In [87]:
good_submissions_df.id.progress_map(get_label_from_submission)


  8%|▊         | 153/2000 [00:24<04:56,  6.24it/s]


 15%|█▌        | 304/2000 [00:49<04:29,  6.28it/s]


 23%|██▎       | 455/2000 [01:13<04:04,  6.31it/s]


 30%|███       | 606/2000 [01:37<03:40,  6.33it/s]


 38%|███▊      | 757/2000 [02:00<03:15,  6.35it/s]


...

 82%|████████▎ | 1650/2000 [04:22<00:55,  6.34it/s]


 90%|████████▉ | 1798/2000 [04:45<00:31,  6.35it/s]


 97%|█████████▋| 1946/2000 [05:08<00:08,  6.34it/s]


100%|██████████| 2000/2000 [05:17<00:00,  6.30it/s]


29410    ESH
16142    YTA
4184     YTA
28267    YTA
7540     YTA
        ... 
9508     YTA
6646     NAH
29158    NAH
6394     YTA
32594    NAH
Name: id, Length: 2000, dtype: object

'YTA'